In [ ]:
import pickle
import sys
from zoneinfo import ZoneInfo
sys.path.append("../")

from dotenv import load_dotenv
load_dotenv()
import geopandas as gpd
import importlib
import logging
import fastsim as fsim
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import lightning.pytorch as pl
import rasterio as rio
from rasterio.plot import show
import seaborn as sns
import shapely
import statsmodels.api as sm
from torch.utils.data import DataLoader

from openbustools import plotting, spatial, standardfeeds
from openbustools.traveltime import data_loader, model_utils
from openbustools.drivecycle import trajectory
from openbustools.drivecycle.physics import conditions, energy, vehicle

In [ ]:
epsg = 32148
timezone = "America/Los_Angeles"
coord_ref_center = [386910,69022]
apply_filter = True
dem_file = Path("..","data","kcm_spatial","usgs10m_dem_32148.tif")
phone_trajectory_folders = [x for x in Path("..","data","kcm_sensor","match_realtime").glob("*") if x.is_dir()]
realtime_folder = Path("..","data","kcm_sensor_realtime")

### Phone, Realtime, Receiver Comparison

In [ ]:
# Create a trajectory for each tracked phone record and its corresponding realtime trip
phone_trajectories = [standardfeeds.get_phone_trajectory(x, timezone=timezone, epsg=epsg, coord_ref_center=coord_ref_center, apply_filter=apply_filter) for x in phone_trajectory_folders]
phone_realtime_trajectories = [standardfeeds.get_realtime_trajectory(x, realtime_folder=realtime_folder, dem_file=dem_file) for x in phone_trajectories]

In [ ]:
# Run fastsim energy analysis for each trajectory
energy_results = []
# Load vehicle
veh = fsim.vehicle.Vehicle.from_vehdb(17)

# Calculate cycle energy consumption
for i, (traj_phone, traj_realtime) in enumerate(zip(phone_trajectories, phone_realtime_trajectories)):
    # Run energy analysis for phone
    cycle_phone = traj_phone.to_fastsim_cycle()
    cycle_phone = fsim.cycle.Cycle.from_dict(fsim.cycle.resample(cycle_phone, new_dt=1))
    sim_drive_phone = fsim.simdrive.SimDrive(cycle_phone, veh)
    sim_drive_phone.sim_drive()
    # Run energy analysis for realtime
    cycle_realtime = traj_realtime.to_fastsim_cycle()
    cycle_realtime = fsim.cycle.Cycle.from_dict(fsim.cycle.resample(cycle_realtime, new_dt=1))
    sim_drive_realtime = fsim.simdrive.SimDrive(cycle_realtime, veh)
    sim_drive_realtime.sim_drive()
    energy_results.append({"phone": (cycle_phone, sim_drive_phone), "realtime": (cycle_realtime, sim_drive_realtime)})

In [ ]:
# Plot comparisons of key metrics over trajectory for each sensor source
efficiencies = []

fig, axes = plt.subplots(len(energy_results), 3, figsize=(20,20))
fig.tight_layout()
for i, res in enumerate(energy_results):
    res_cycle_phone, res_simdrive_phone = res["phone"]
    res_cycle_realtime, res_simdrive_realtime = res["realtime"]
    efficiencies.append((1 / res_simdrive_phone.electric_kwh_per_mi, 1 / res_simdrive_realtime.electric_kwh_per_mi))
    ax = axes[i,0]
    ax.plot(res_simdrive_phone.cyc.time_s, res_simdrive_phone.mph_ach, label="Phone")
    ax.plot(res_simdrive_realtime.cyc.time_s, res_simdrive_realtime.mph_ach, label="Realtime")
    ax = axes[i,1]
    ax.plot(res_simdrive_phone.cyc.time_s, res_simdrive_phone.ess_kw_out_ach, label="Phone")
    ax.plot(res_simdrive_realtime.cyc.time_s, res_simdrive_realtime.ess_kw_out_ach, label="Realtime")
    ax = axes[i,2]
    ax.plot(res_simdrive_phone.cyc.time_s, res_simdrive_phone.soc, label="Phone")
    ax.plot(res_simdrive_realtime.cyc.time_s, res_simdrive_realtime.soc, label="Realtime")
axes[0,0].set_title("Phone vs Realtime Speed [MPH]")
axes[0,0].set_xlabel("Time [s]")
axes[0,0].set_ylabel("Speed [MPH]")
axes[0,1].set_title("Phone vs Realtime ESS Power Out [kW]")
axes[0,1].set_xlabel("Time [s]")
axes[0,1].set_ylabel("Total Power [kW]")
axes[0,2].set_title("Phone vs Realtime Battery SoC [%]")
axes[0,2].set_xlabel("Time [s]")
axes[0,2].set_ylabel("SoC [%]")
axes[0,2].legend()
plt.show()
print(f"Avg. Efficiency: {np.mean([x[0] for x in efficiencies]):.2f} phone, {np.mean([x[1] for x in efficiencies]):.2f} realtime, {np.sqrt(np.mean([(x-y)**2 for x,y in efficiencies])):.2f} mi/kWh RMSE")

In [ ]:
# power_components = {
#     "accel_kw": sim_drive.accel_kw,
#     "rr_kw": sim_drive.rr_kw,
#     "ess_loss_kw": sim_drive.ess_loss_kw,
#     "ascent_kw": sim_drive.ascent_kw,
#     "drag_kw": sim_drive.drag_kw,
#     "aux_in_kw": sim_drive.aux_in_kw,
# }
# fig, ax = plt.subplots(6, 1, figsize=(9,9))
# fig.tight_layout()
# i=0
# for name, power in power_components.items():
#     ax[i].plot(cyc.time_s, power, label=name)
#     ax[i].set_xlabel('Time [s]')
#     ax[i].set_ylabel(name)
#     i+=1

### Mean Realtime, Predicted Comparison

### Expand Predicted to Full Network

### Phone Baseline Compared to Realtime

In [ ]:
importlib.reload(trajectory)
importlib.reload(energy)
importlib.reload(standardfeeds)
importlib.reload(vehicle)
importlib.reload(spatial)

all_phone_intensities = []
all_realtime_intensities = []

# List of phone trajectories available that have matching realtime data
phone_trajectory_folders = [x for x in Path("..","data","kcm_sensor","match_realtime").glob("*") if x.is_dir()]

# Build trajectory for each phone recording
for phone_trajectory_folder in phone_trajectory_folders:
    # Load phone data
    metadata_phone, data_phone = standardfeeds.combine_phone_sensors(phone_trajectory_folder, "America/Los_Angeles")
    data_phone.loc[data_phone['speed']<0, 'speed'] = np.nan
    data_phone.loc[data_phone['bearing']<0, 'bearing'] = np.nan
    data_phone = data_phone.bfill()
    # Create trajectory
    phone_traj = trajectory.Trajectory(
        point_attr={
            "lon": data_phone.longitude.to_numpy(),
            "lat": data_phone.latitude.to_numpy(),
            "measured_elev_m": data_phone.altitudeAboveMeanSeaLevel.to_numpy(),
            "measured_speed_m_s": data_phone.speed.to_numpy(),
            "measured_bear_d": data_phone.bearing.to_numpy(),
            "measured_accel_m_s2": np.diff(data_phone.speed.to_numpy(), prepend=0), # Can be accelerometer or speed derivative
            "measured_theta_d": data_phone.pitch.to_numpy() - np.mean(data_phone.pitch.to_numpy()),
            "cumul_time_s": data_phone.cumul_time_s.to_numpy(),
        },
        traj_attr=metadata_phone,
        coord_ref_center=[386910, 69022],
        epsg=32148,
        apply_filter=False
    )
    # Create drivecycle
    phone_cycle = phone_traj.measured_to_drivecycle()
    energy_model = energy.EnergyModel(vehicle.TransitBus(), conditions.AmbientConditions())
    phone_cycle_energy_df = energy_model.getEnergyDataFrame(phone_cycle)
    all_phone_intensities.append(energy_model.calcIntensity(phone_cycle)*1.6)


    # Load corresponding realtime data
    data_realtime = pd.read_pickle(Path("..", "data", "kcm_sensor_realtime", f"{phone_traj.traj_attr['t_day']}.pkl"))
    # Pull the phone trajectory
    data_realtime = data_realtime[data_realtime['vehicleid'].astype(int).astype(str) == phone_traj.traj_attr['veh_id']]
    data_realtime = data_realtime[data_realtime['locationtime'].astype(int) >= phone_traj.traj_attr['start_epoch']]
    data_realtime = data_realtime[data_realtime['locationtime'].astype(int) <= phone_traj.traj_attr['end_epoch']]
    metadata_realtime = metadata_phone.copy()
    metadata_realtime.update({
        "start_epoch": data_realtime['locationtime'].iloc[0].astype(int),
        "end_epoch": data_realtime['locationtime'].iloc[-1].astype(int)
    })
    # Create trajectory
    realtime_traj = trajectory.Trajectory(
        point_attr={
            "lon": data_realtime.lon.to_numpy(),
            "lat": data_realtime.lat.to_numpy(),
            "cumul_time_s": (data_realtime.locationtime - metadata_phone['start_epoch']).to_numpy(),
        },
        traj_attr=metadata_realtime,
        coord_ref_center=[386910, 69022],
        epsg=32148,
        dem_file=Path("..", "data", "kcm_spatial","usgs10m_dem_32148.tif"),
        apply_filter=False
    )
    # Create drivecycle
    realtime_cycle = realtime_traj.gps_to_drivecycle()
    energy_model = energy.EnergyModel(vehicle.TransitBus(), conditions.AmbientConditions())
    realtime_cycle_energy_df = energy_model.getEnergyDataFrame(realtime_cycle)
    all_realtime_intensities.append(energy_model.calcIntensity(realtime_cycle)*1.6)

In [ ]:
print("Phone")
print(f"Mean: {np.mean(all_phone_intensities):.2f} kWh/mi")
print(f"Std: {np.std(all_phone_intensities):.2f} kWh/mi")
print()
print("Realtime") 
print(f"Mean: {np.mean(all_realtime_intensities):.2f} kWh/mi")
print(f"Std: {np.std(all_realtime_intensities):.2f} kWh/mi")
print()
print(f"RMSE: {np.sqrt(np.mean((np.array(all_phone_intensities) - np.array(all_realtime_intensities))**2)):.2f} kWh/mi")

In [ ]:
realtime_cycle_energy_df['Source'] = 'Realtime'
realtime_cycle_energy_df['cumul_time_s'] = realtime_cycle_energy_df['Time'].cumsum()
phone_cycle_energy_df['Source'] = 'Phone'
phone_cycle_energy_df['cumul_time_s'] = phone_cycle_energy_df['Time'].cumsum()
rt_df = realtime_cycle_energy_df[['Source','cumul_time_s','Velocity','Acceleration','Time','Distance']]
ph_df = phone_cycle_energy_df[['Source','cumul_time_s','Velocity','Acceleration','Time','Distance']]
plot_df = pd.concat([rt_df, ph_df], axis=0)
fig = plotting.formatted_trajectory_lineplot(plot_df, "Phone and Realtime Trajectories")

In [ ]:
importlib.reload(plotting)
realtime_cycle_energy_df['Source'] = 'Realtime'
realtime_cycle_energy_df['cumul_time_s'] = realtime_cycle_energy_df['Time'].cumsum()
phone_cycle_energy_df['Source'] = 'Phone'
phone_cycle_energy_df['cumul_time_s'] = phone_cycle_energy_df['Time'].cumsum()
rt_df = realtime_cycle_energy_df[['Source','cumul_time_s','F_aero','F_grav','F_roll','F_acc','P_tot']]
ph_df = phone_cycle_energy_df[['Source','cumul_time_s','F_aero','F_grav','F_roll','F_acc','P_tot']]
plot_df = pd.concat([rt_df, ph_df], axis=0)
fig = plotting.formatted_forces_lineplot(plot_df, "Phone and Realtime Forces")

### Phone Baseline Compared to Resampled + Modeled

In [ ]:
importlib.reload(trajectory)
importlib.reload(energy)
importlib.reload(standardfeeds)
importlib.reload(vehicle)
importlib.reload(spatial)
importlib.reload(pl)
importlib.reload(data_loader)


logging.getLogger("pytorch_lightning").setLevel(logging.CRITICAL)
all_phone_intensities = []
all_realtime_intensities = []

# List of phone trajectories available that have matching realtime data
phone_trajectory_folders = [x for x in Path("..","data","kcm_sensor","match_realtime").glob("*") if x.is_dir()]

# Build trajectory for each phone recording
for phone_trajectory_folder in phone_trajectory_folders:
    # Load phone data
    metadata_phone, data_phone = standardfeeds.combine_phone_sensors(phone_trajectory_folder, "America/Los_Angeles")
    data_phone.loc[data_phone['speed']<0, 'speed'] = np.nan
    data_phone.loc[data_phone['bearing']<0, 'bearing'] = np.nan
    data_phone = data_phone.bfill()
    # Create trajectory
    phone_traj = trajectory.Trajectory(
        point_attr={
            "lon": data_phone.longitude.to_numpy(),
            "lat": data_phone.latitude.to_numpy(),
            "measured_elev_m": data_phone.altitudeAboveMeanSeaLevel.to_numpy(),
            "measured_speed_m_s": data_phone.speed.to_numpy(),
            "measured_bear_d": data_phone.bearing.to_numpy(),
            "measured_accel_m_s2": np.diff(data_phone.speed.to_numpy(), prepend=0), # Can be accelerometer or speed derivative
            "measured_theta_d": data_phone.pitch.to_numpy() - np.mean(data_phone.pitch.to_numpy()),
            "cumul_time_s": data_phone.cumul_time_s.to_numpy(),
        },
        traj_attr=metadata_phone,
        coord_ref_center=[386910, 69022],
        epsg=32148,
        apply_filter=False
    )
    # Create drivecycle
    phone_cycle = phone_traj.measured_to_drivecycle()
    energy_model = energy.EnergyModel(vehicle.TransitBus(), conditions.AmbientConditions())
    phone_cycle_energy_df = energy_model.getEnergyDataFrame(phone_cycle)
    all_phone_intensities.append(energy_model.calcIntensity(phone_cycle)*1.6)


    # Load corresponding realtime data
    data_realtime = pd.read_pickle(Path("..", "data", "kcm_sensor_realtime", f"{phone_traj.traj_attr['t_day']}.pkl"))
    # Pull the phone trajectory
    data_realtime = data_realtime[data_realtime['vehicleid'].astype(int).astype(str) == phone_traj.traj_attr['veh_id']]
    data_realtime = data_realtime[data_realtime['locationtime'].astype(int) >= phone_traj.traj_attr['start_epoch']]
    data_realtime = data_realtime[data_realtime['locationtime'].astype(int) <= phone_traj.traj_attr['end_epoch']]
    metadata_realtime = metadata_phone.copy()
    metadata_realtime.update({
        "start_epoch": data_realtime['locationtime'].iloc[0].astype(int),
        "end_epoch": data_realtime['locationtime'].iloc[-1].astype(int)
    })
    # Create trajectory
    realtime_traj = trajectory.Trajectory(
        point_attr={
            "lon": data_realtime.lon.to_numpy(),
            "lat": data_realtime.lat.to_numpy(),
            "cumul_time_s": (data_realtime.locationtime - metadata_phone['start_epoch']).to_numpy(),
        },
        traj_attr=metadata_realtime,
        coord_ref_center=[386910, 69022],
        epsg=32148,
        dem_file=Path("..", "data", "kcm_spatial","usgs10m_dem_32148.tif"),
        apply_filter=False,
        resample_len=100
    )

    # Update the trajectory with predicted travel times
    model = model_utils.load_model("../logs/", "kcm", "GRU", 0)
    realtime_traj.update_predicted_time(model)

    # Create drivecycle
    realtime_cycle = realtime_traj.gps_to_drivecycle()
    energy_model = energy.EnergyModel(vehicle.TransitBus(), conditions.AmbientConditions())
    realtime_cycle_energy_df = energy_model.getEnergyDataFrame(realtime_cycle)
    all_realtime_intensities.append(energy_model.calcIntensity(realtime_cycle)*1.6)

In [ ]:
print("Phone")
print(f"Mean: {np.mean(all_phone_intensities):.2f} kWh/mi")
print(f"Std: {np.std(all_phone_intensities):.2f} kWh/mi")
print()
print("Realtime")
print(f"Mean: {np.mean(all_realtime_intensities):.2f} kWh/mi")
print(f"Std: {np.std(all_realtime_intensities):.2f} kWh/mi")
print()
print(f"RMSE: {np.sqrt(np.mean((np.array(all_phone_intensities) - np.array(all_realtime_intensities))**2)):.2f} (kWh/mi)")

In [ ]:
realtime_cycle_energy_df['Source'] = 'Realtime'
realtime_cycle_energy_df['cumul_time_s'] = realtime_cycle_energy_df['Time'].cumsum()
phone_cycle_energy_df['Source'] = 'Phone'
phone_cycle_energy_df['cumul_time_s'] = phone_cycle_energy_df['Time'].cumsum()
rt_df = realtime_cycle_energy_df[['Source','cumul_time_s','Velocity','Acceleration','Time','Distance']]
ph_df = phone_cycle_energy_df[['Source','cumul_time_s','Velocity','Acceleration','Time','Distance']]
plot_df = pd.concat([rt_df, ph_df], axis=0)
fig = plotting.formatted_trajectory_lineplot(plot_df, "Phone and Realtime Trajectories")

### Compare Phone Sensor Kinematics

In [ ]:
# fig, axes = plt.subplots(1,1,figsize=(8,4))
# phone_traj.gdf['calc_dist_m'].cumsum().plot(ax=axes)
# phone_traj.gdf['measured_speed_m_s'].cumsum().plot(ax=axes)
# phone_traj.gdf['measured_accel_m_s2'].cumsum().cumsum().plot(ax=axes)
# axes.legend(['gps','integrated speedometer','integrated accelerometer'])
# axes.set_title("Total Distance (m)")

In [ ]:
# fig, axes = plt.subplots(1,1,figsize=(8,4))
# phone_traj.gdf['calc_dist_m'].plot(ax=axes)
# phone_traj.gdf['measured_speed_m_s'].plot(ax=axes)
# phone_traj.gdf['measured_accel_m_s2'].cumsum().plot(ax=axes)
# axes.legend(['rate of change gps','speedometer','integrated accelerometer'])
# axes.set_title("Velocity (m/s)")

In [ ]:
# fig, axes = plt.subplots(1,1,figsize=(8,4))
# phone_traj.gdf['calc_dist_m'].diff().clip(-1,1).plot(ax=axes)
# phone_traj.gdf['measured_speed_m_s'].diff().plot(ax=axes)
# phone_traj.gdf['measured_accel_m_s2'].plot(ax=axes)
# axes.legend(['rate of change gps','rate of change speedometer','accelerometer'])
# axes.set_title("Acceleration (m/s^2)")

### Distance Along GTFS Shape

In [ ]:
# # GTFS shapes
# shape_lookup = standardfeeds.get_gtfs_shapes_lookup(f"../data/kcm_gtfs/{static_date}/")
# shapes = standardfeeds.get_gtfs_shapes(f"../data/kcm_gtfs/{static_date}/").to_crs("EPSG:32148")
# shapes.plot()

# route_ids = pd.unique(data_gtfs[(data_gtfs['route_short_name']==short_name) & (data_gtfs['direction_id']==0)].route_id)
# phone_shape = shapes[(shapes['route_id'].isin(route_ids)) & (shapes['direction_id']==0) & (shapes['service_id']==21133)]

# # Get one shape to work with
# sample_service_id, sample_route_id, sample_direction_id = data_gtfsrt.groupby(['service_id','route_id','direction_id']).count().index[0]
# print(sample_service_id, sample_route_id, sample_direction_id)

# # GTFS-RT
# sample_realtime = data_gtfsrt[(data_gtfsrt['service_id']==sample_service_id) & (data_gtfsrt['route_id']==sample_route_id) & (data_gtfsrt['direction_id']==sample_direction_id)].copy()

# # Shape
# sample_shape = shapes[(shapes['service_id']==sample_service_id) & (shapes['route_id']==sample_route_id) & (shapes['direction_id']==sample_direction_id)].copy()
# sample_shape.plot()

# # Get distance along shape
# sample_realtime['dist_along_line'] = sample_realtime['geometry'].apply(lambda pt: shapely.line_locate_point(sample_shape.geometry, pt))
# # sample_static['dist_along_line'] = sample_static['geometry'].apply(lambda pt: shapely.line_locate_point(sample_shape.geometry, pt))

# # Also get a timestamp column on the samples
# sample_realtime['t'] = pd.to_datetime(sample_realtime['locationtime'], unit='s')
# sample_realtime = sample_realtime.set_index('t')